In [ ]:
#requirements
import numpy as np
import time
import sklearn as sk
import MLRNN as MLR
from copy import deepcopy
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler as normalize
from sklearn.model_selection import train_test_split as tts

#also torch, xgboost, catboost see below

# User inputs (check this section then run all cells)

In [ ]:
import importlib
if importlib.util.find_spec('torch') is None:
    raise ImportError("MLR is implemented in torch here! => conda install -c pytorch pytorch")
xgboost_available = importlib.util.find_spec('xgboost') is not None #conda install -c conda-forge xgboost
catboost_available = importlib.util.find_spec('catboost') is not None #conda install -c conda-forge catboost
#else excluded from the benchmark

In [ ]:
input_repository = "../preprocessed_datasets/"
output_repository = "outputs/"
HPO_benchmark_output_file = "HPO_benchmark.csv"

In [ ]:
benchmark_datasets = np.arange(16) #16 for regression
benchmark_seeds = 10

In [ ]:
n_trials = 50 #Trials
cvs = 5 #Trial Kfold
reps = 5 #estimators for Bagging/Ensemble 

In [ ]:
run_HPO_benchmark = True

# Load datasets

In [ ]:
def dataset_loader(dataset_id, name, repository):
    return np.load(repository + name + str(dataset_id) + ".npy")
def prepare_dataset(dataset, train_size = 0.8, seed= False):
    kwargs = {}
    if seed or type(seed) == type(0):
        kwargs["random_state"] = seed
    X, y = dataset[:, :-1], dataset[:, -1]
    X = normalize().fit_transform(X)
    X_train, X_test, y_train, y_test = tts(X, y, train_size = train_size, **kwargs)
    return X_train, X_test, y_train, y_test
def get_dataset(dataset_id, name, repository, train_size = 0.8, seed = False):
    return prepare_dataset(dataset_loader(dataset_id, name, repository), train_size = train_size, seed = seed)

# Write results

In [ ]:
def write_results(results, output_file, output_repository, metrics = ["R2"]):
    import os
    if output_file not in os.listdir(output_repository):
        with open(output_repository + output_file, "w") as file:
            file.write(",".join(["id","dataset","seed","category", "method","time"]+metrics))
            file.close()
    with open(output_repository + output_file, "a") as file:
        file.write("\n"+",".join(map(str,results)))
        file.close()

# Method evaluation

In [ ]:
def run_HPO_experiment(methods, datasets, input_name, input_repository, output_file, output_repository, seeds = 10, n_trials = 100, cvs = 10, reps=10):
    metrics = ["R2","Bagging-R2","Ensemble-R2"]
    for dataset_id in datasets:
        for seed in range(seeds):
            X_train, X_test, y_train, y_test = get_dataset(dataset_id, input_name, input_repository, train_size = 0.8, seed = seed)
            for method_name in methods:
                method_category = method_name
                exp_id = str(dataset_id)+'_'+str(seed)+"_"+str(method_category)+"_"+str(method_name)
                start_time = time.time()
                results = run_HPO(method_name, X_train, y_train, X_test, y_test, n_trials = n_trials, cvs = cvs, reps = 10)
                end_time = time.time() - start_time
                
                result_line = [exp_id, dataset_id, seed, method_category, method_name, end_time]+results
                write_results(result_line, output_file, output_repository, metrics = metrics)

In [ ]:
import time
import optuna
from functools import partial
def get_trials(objective, n_trials = 100):
    study = optuna.create_study( directions=["maximize"])
    study.optimize(objective, n_trials=n_trials, timeout = 6000)
    params = [trial.params for trial in study.trials]
    return [params[i] for i in np.argsort([trial.value for trial in study.trials])][::-1]
def get_perfs(model, params, X_train, y_train, X_test, y_test, reps = 10):
    best_params = params[0]
    mean_perf = 0
    bagging_preds = 0
    for rep in range(reps):
        best_params.update({"random_state":rep})
        reg = model(**best_params)
        try:
            pred = reg.fit(X_train, y_train).predict(X_test)
        except:
            pred = np.zeros(len(X_test))
        mean_perf += r2_score(y_test, pred)/reps
        bagging_preds += pred/reps
        if model == "MLR":
            reg.delete_model_weights()
            del reg
    bagging_perf = r2_score(y_test, bagging_preds)
    ensemble_preds = 0
    for rep in range(reps):
        best_params = params[rep]
        best_params.update({"random_state":rep})
        reg = model(**best_params)
        try:
            pred = reg.fit(X_train, y_train).predict(X_test)
        except:
            pred = np.zeros(len(X_test))
        ensemble_preds += pred/reps
        if model == "MLR":
            reg.delete_model_weights()
            del reg
    ensemble_perf = r2_score(y_test, ensemble_preds)
    return [mean_perf, bagging_perf, ensemble_perf]
def run_HPO(method_name, X_train, y_train, X_test, y_test, n_trials= 100, cvs = 10, reps = 10):
    objective = partial(objectives[method_name], X_train=X_train, y_train= y_train, cvs = cvs)
    params_process = params_processes[method_name]
    model = models[method_name]
    params = map(params_process, get_trials(objective, n_trials))
    return get_perfs(model, list(params), X_train, y_train, X_test, y_test, reps = reps)

# Methods

In [ ]:
def MLR_objective(trial, X_train, y_train, cvs = 10):
    from sklearn.model_selection import KFold
    import MLRNN as MLR
    min_bs = float(16/X_train.shape[0])
    parameters = {"max_runtime" : 6}

    parameters["depth"] = trial.suggest_int("depth", 1, 5, log=False)
    parameters["width"]= trial.suggest_int("width", 16, 4096, log=True)
    parameters["ridge_init"] = trial.suggest_float("ridge_init", 1e-1, 1e7, log=True)
    
    w = parameters["width"]
    min_lr, max_lr  = np.max([np.min([1/w * 1e-2, 1e-1]), 1e-5]),np.max([np.min([1/w * 1e1, 1e-1]), 1e-5])
    parameters["learning_rate"] = trial.suggest_float("learning_rate", min_lr, max_lr, log=True)
    parameters["max_iter"] = int(np.max([np.min([(w*1e-5)**(-0.5), 10]), 300]))
    perfs = 0.
    kf = KFold(n_splits=cvs,random_state=None, shuffle=True)
    for train_index, valid_index in kf.split(X_train):
        X_t, X_v, y_t, y_v = X_train[train_index], X_train[valid_index], y_train[train_index], y_train[valid_index]
        reg = MLR.MLRNNRegressor(**parameters)
        try:
            perfs += reg.fit(X_t, y_t).score(X_v, y_v) / cvs
        except:
            reg.delete_model_weights()
            del reg
            return perfs
        reg.delete_model_weights()
        del reg
    return perfs

In [ ]:
def XGBoost_objective(trial, X_train, y_train, cvs = 10):
    from xgboost import XGBRegressor as XGB
    params = {"objective" :'reg:squarederror'}
    params["learning_rate"] = trial.suggest_float("learning_rate", 1e-7 ,1, log = True )
    params["max_depth"] = trial.suggest_int("max_depth", 1, 10)
    params["subsample"] = trial.suggest_float("subsample", 0.2, 1)
    params["colsample_bytree"] = trial.suggest_float("colsample_bytree", 0.2, 1)
    params["colsample_bylevel"] = trial.suggest_float("colsample_bylevel", 0.2, 1)
    params["min_child_weight"] = trial.suggest_float("min_child_weight", 1e-16,1e5, log = True)
    params["reg_alpha"] = trial.suggest_float("reg_alpha", 1e-16, 1e2, log = True)
    params["reg_lambda"] = trial.suggest_float("reg_lambda", 1e-16, 1e2, log = True)
    params["gamma"] = trial.suggest_float("gamma", 1e-16, 1e2, log = True)
    from sklearn.model_selection import KFold
    perfs = 0.
    kf = KFold(n_splits=cvs,random_state=None, shuffle=True)
    for train_index, valid_index in kf.split(X_train):
        X_t, X_v, y_t, y_v = X_train[train_index], X_train[valid_index], y_train[train_index], y_train[valid_index]
        reg = XGB(**params)
        perfs += reg.fit(X_t, y_t).score(X_v, y_v)/cvs
    return perfs

In [ ]:
def CATBoost_objective(trial, X_train, y_train, cvs = 10):
    from catboost import CatBoostRegressor as CAT
    params = {"logging_level":'Silent'}
    params["learning_rate"] = trial.suggest_float("learning_rate", 1e-5, 1, log = True)
    params["random_strength"] = trial.suggest_int("random_strength", 1, 20)
    params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 1)
    params["l2_leaf_reg"] = trial.suggest_float("l2_leaf_reg", 1, 10, log=True)
    params["leaf_estimation_iterations"] = trial.suggest_int("leaf_estimation_iterations", 1, 20)
    from sklearn.model_selection import KFold
    perfs = 0.
    kf = KFold(n_splits=cvs,random_state=None, shuffle=True)
    for train_index, valid_index in kf.split(X_train):
        X_t, X_v, y_t, y_v = X_train[train_index], X_train[valid_index], y_train[train_index], y_train[valid_index]
        reg = CAT(**params)
        perfs += reg.fit(X_t, y_t).score(X_v, y_v)/cvs
    return perfs

In [ ]:
def RF_objective(trial, X_train, y_train, cvs = 10):
    from sklearn.ensemble import RandomForestRegressor as RF
    params = {"n_estimators":100}
    params["max_features"] = trial.suggest_categorical("max_features", ['auto', 'sqrt', "log2"])
    params["max_depth"] = trial.suggest_int("max_depth", 2, 100, log = True)
    params["max_leaf_nodes"] = trial.suggest_int("max_leaf_nodes", 2, 1024, log = True)
    params["min_samples_leaf"] = trial.suggest_int("min_samples_leaf", 1, 16, log = True)
    params["bootstrap"] = trial.suggest_categorical("bootstrap", [True, False])
    params["max_samples"] = trial.suggest_float("max_samples", 0.05, 1.)
    from sklearn.model_selection import KFold
    perfs = 0.
    kf = KFold(n_splits=cvs,random_state=None, shuffle=True)
    for train_index, valid_index in kf.split(X_train):
        X_t, X_v, y_t, y_v = X_train[train_index], X_train[valid_index], y_train[train_index], y_train[valid_index]
        reg = RF(**params)
        perfs += reg.fit(X_t, y_t).score(X_v, y_v)/cvs
    return perfs

In [ ]:
def MLR_params_process(params):
    parameters = {"max_runtime" : 6}
    parameters.update(params)
    w = parameters["width"]
    parameters["max_iter"] = int(np.max([np.min([(w*1e-5)**(-0.5), 10]), 300]))
    return parameters
def CATBoost_params_process(params):
    parameters = {"logging_level":'Silent'}
    parameters.update(params)
    return parameters
def XGBoost_params_process(params):
    parameters = {"objective" :'reg:squarederror'}
    parameters.update(params)
    return parameters
def RF_params_process(params):
    parameters = {"n_estimators":100}
    parameters.update(params)
    return parameters

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF
methods = ["MLR","RF"]
params_processes = {"MLR":MLR_params_process,"RF":RF_params_process}
objectives= {"MLR":MLR_objective,"RF":RF_objective}
models = {"MLR":MLR.MLRNNRegressor,"RF":RF}
if xgboost_available: 
    methods.append("XGB")
    from xgboost import XGBRegressor as XGB
    params_processes["XGB"] = XGBoost_params_process
    objectives["XGB"] = XGBoost_objective
    models["XGB"] = XGB
if catboost_available: 
    methods.append("CAT")
    from catboost import CatBoostRegressor as CAT
    params_processes["CAT"] = CATBoost_params_process
    objectives["CAT"] = CATBoost_objective
    models["CAT"] = CAT

# Run experiments

In [ ]:
if run_HPO_benchmark:
    input_name = "regression"
    run_HPO_experiment(methods, benchmark_datasets, input_name, input_repository, HPO_benchmark_output_file, output_repository, seeds = benchmark_seeds, n_trials = n_trials, cvs = cvs, reps = reps)

# Quick results review

In [1]:
output_repository = "outputs/"
HPO_benchmark_output_file = "HPO_benchmark.csv"

In [2]:
import pandas as pd
import numpy as np
benchmark_seeds = 10
df_results = pd.read_csv(output_repository+HPO_benchmark_output_file)

In [3]:
datasets_df = df_results.groupby('dataset').sum()
n_methods = len(set(df_results['method'].values))
valid_datasets = datasets_df[datasets_df["seed"] == np.sum(np.arange(benchmark_seeds)) * n_methods].index.values
valid_datasets

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [4]:
df_results_complete = df_results[np.isin(df_results["dataset"].values, valid_datasets)]
global_results_cols = ['method']
detailed_results_cols = ["dataset",'method']
metrics = ['R2','Bagging-R2','Ensemble-R2']

In [5]:
df_global = df_results_complete.groupby(global_results_cols).mean()[metrics]
df_global #MLR>CAT>RF>XGB

,R2,Bagging-R2,Ensemble-R2
method,,,
CAT,0.728598,0.730836,0.734963
MLR,0.724706,0.743147,0.744805
RF,0.728637,0.729995,0.734015
XGB,0.716487,0.722311,0.728117


In [6]:
df_detailed = df_results_complete.groupby(detailed_results_cols).mean()[metrics]
df_detailed 

R2  Bagging-R2  Ensemble-R2
dataset method                                   
0       CAT     0.165442    0.176159     0.215407
        MLR     0.276468    0.375376     0.385346
        RF      0.262384    0.266022     0.299112
        XGB     0.153888    0.169309     0.222053
1       CAT     0.540686    0.542833     0.539904
        MLR     0.453167    0.483092     0.485849
        RF      0.481252    0.484735     0.490580
        XGB     0.487532    0.501005     0.502446
2       CAT     0.977607    0.979298     0.979864
        MLR     0.988100    0.992130     0.992583
        RF      0.973944    0.974580     0.973985
        XGB     0.976746    0.982232     0.981912
3       CAT     0.984959    0.985541     0.984998
        MLR     0.974581    0.976805     0.976137
        RF      0.977637    0.977712     0.977975
        XGB     0.986451    0.987752     0.987369
4       CAT     0.686440    0.689125     0.689270
        MLR     0.674679    0.683547     0.685981
        RF      0.675938    0.678435     0.678381
        XGB     0.670305    0.676728     0.680336
5       CAT     0.847338    0.847544     0.847302
        MLR     0.837688    0.842875     0.843161
        RF      0.846248    0.846653     0.846801
        XGB     0.835008    0.838685     0.835357
6       CAT     0.908897    0.909302     0.909531
        MLR     0.893275    0.898225     0.898135
        RF      0.906376    0.907102     0.906074
        XGB     0.904565    0.906114     0.907064
7       CAT     0.521618    0.521629     0.522120
        MLR     0.521607    0.522478     0.522755
        RF      0.522117    0.522243     0.522389
        XGB     0.517668    0.517820     0.517891
8       CAT     0.924393    0.926091     0.926268
        MLR     0.902793    0.913791     0.913302
        RF      0.911838    0.912475     0.910836
        XGB     0.916222    0.921152     0.918624

#### Comments: 
MLR outperforms others on Dataset 0 by a lot.

CAT outperforms others on Dataset 1 by a lot.

For the other datasets, the results seem quite comparable so far...

In [7]:
df_zero_excluded = df_detailed.reset_index()
df_zero_excluded = df_zero_excluded[df_zero_excluded["dataset"] != 0]
df_zero_excluded = df_zero_excluded.groupby("method").mean()[metrics]
df_zero_excluded #CAT>XGB>RF>MLR

,R2,Bagging-R2,Ensemble-R2
method,,,
CAT,0.798992,0.800170,0.799907
MLR,0.780736,0.789118,0.789738
RF,0.786919,0.787992,0.788378
XGB,0.786812,0.791436,0.791375


In [8]:
df_one_excluded = df_detailed.reset_index()
df_one_excluded = df_one_excluded[df_one_excluded["dataset"] != 1]
df_one_excluded = df_one_excluded.groupby("method").mean()[metrics]
df_one_excluded #MLR>RF>CAT>XGB

,R2,Bagging-R2,Ensemble-R2
method,,,
CAT,0.752087,0.754336,0.759345
MLR,0.758649,0.775653,0.777175
RF,0.759560,0.760653,0.764444
XGB,0.745107,0.749974,0.756326


In [9]:
df_both_excluded = df_detailed.reset_index()
df_both_excluded = df_both_excluded[np.logical_not(np.isin(df_both_excluded["dataset"].values,[0,1] ))]
df_both_excluded = df_both_excluded.groupby("method").mean()[metrics]
df_both_excluded #Not that conclusive

,R2,Bagging-R2,Ensemble-R2
method,,,
CAT,0.835893,0.836933,0.83705
MLR,0.827532,0.832836,0.83315
RF,0.830586,0.831314,0.83092
XGB,0.829566,0.832926,0.83265
